##### Генерируются последовательности, которые состоят из цифр (от 0 до 9) и задаются следующим образом:
##### $x$ - последовательность случайных цифр
##### $y_1 = x_1$
##### $y_i = x_i + x_1$
##### Если $y >= 10,$ то $y_i = y_i - 10$
##### Модель рекуррентной нейронной сети предсказвает $y_i$ по $x_i$.
##### Используются: $RNN,$ $LSTM,$ $GRU$

In [ ]:
from io import open
import re
import torch
import time
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn

import warnings
warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
LENGTH_SEQUENCE = 10
LEARNING_RATE = 0.7
NUM_EPOCHS = 60
BATCH_SIZE = 512
INPUT_DIM = 10

##### Генерируем одно наблюдение $x$ и $y$ длины LENGTH_SEQUENCE

In [ ]:
def generate_sequences(length = LENGTH_SEQUENCE):
    x = torch.randint(0, 10, (length,))

    y = torch.cat((x[0].unsqueeze(dim=0), torch.add(x[0], x[1:])))

    for i in y:
        if i >= 10:
            i -= 10

    return x.unsqueeze(dim=0), y.unsqueeze(dim=0)

##### Создаем тренировочный, тестовый и валидационный датасет

In [ ]:
def create_dataset(number_sequences = 10_000, length_sequence = LENGTH_SEQUENCE, test_size = 0.2, val_size = 0.1, BATCH_SIZE = BATCH_SIZE):
    x_train, y_train = generate_sequences(length_sequence)
    for _ in range(number_sequences - 1):
        x, y = generate_sequences(length_sequence)
        x_train = torch.cat((x_train, x))
        y_train = torch.cat((y_train, y))

    x_test, y_test = generate_sequences(length_sequence)
    for _ in range(round(number_sequences * test_size) - 1):
        x, y = generate_sequences(length_sequence)
        x_test = torch.cat((x_test, x))
        y_test = torch.cat((y_test, y))

    x_val, y_val = generate_sequences(length_sequence)
    for _ in range(round(number_sequences * val_size) - 1):
        x, y = generate_sequences(length_sequence)
        x_val = torch.cat((x_val, x))
        y_val = torch.cat((y_val, y))

    train_dataset = TensorDataset(x_train, y_train)
    test_dataset = TensorDataset(x_test, y_test)
    val_dataset = TensorDataset(x_val, y_val)

    train = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

    return train, test, val

In [ ]:
class Model(torch.nn.Module):

    def __init__(self, model, embed_dim, hidden_dim, layer_dim):
        super().__init__()
        self.embed = nn.Embedding(INPUT_DIM, embed_dim)
        self.model = model(embed_dim, hidden_dim, layer_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, INPUT_DIM)

    def forward(self, sentence, state=None):
        embed = self.embed(sentence)
        o, _ = self.model(embed)
        return self.linear(o)

In [ ]:
class Training(object):
    def __init__(self, model, loss_fn, optimizer):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer

    def train(self, train, test):
        for epoch in range(1, NUM_EPOCHS + 1):
            train_loss, train_accuracy, iter_num = .0, .0, .0
            start_epoch_time = time.time()
            self.model.train().to(device)
            for x, y in train:
                x = x.to(device)
                y = y.view(1, -1).squeeze().to(device)

                self.optimizer.zero_grad()

                out = self.model.forward(x).view(-1, INPUT_DIM)

                loss = self.loss_fn(out, y)
                train_loss += loss.item()

                batch_accuracy = (out.argmax(dim=1) == y)
                train_accuracy += batch_accuracy.sum().item() / batch_accuracy.shape[0]

                loss.backward()
                self.optimizer.step()
                iter_num += 1
            if (epoch < 2) | (epoch % 10 == 0):
                print(f"Epoch: {epoch}, loss: {train_loss:.4f}, acc: " f"{train_accuracy / iter_num:.4f}", end=" | ")

            test_loss, test_accuracy, iter_num = .0, .0, .0
            self.model.eval().to(device)
            for x, y in test:
                x = x.to(device)
                y = y.view(1, -1).squeeze().to(device)

                out = self.model.forward(x).view(-1, INPUT_DIM)

                loss = self.loss_fn(out, y)
                test_loss += loss.item()

                batch_accuracy = (out.argmax(dim=1) == y)
                test_accuracy += batch_accuracy.sum().item() / batch_accuracy.shape[0]
                iter_num += 1
            if (epoch < 2) | (epoch % 10 == 0):
                print(f"test loss: {test_loss:.4f}, test acc: {test_accuracy / iter_num:.4f} | " f"{time.time() - start_epoch_time:.2f} sec.")

In [ ]:
train, test, val = create_dataset()

In [ ]:
models = {
        "RNN": [nn.RNN, 32, 128, 5],
        "LSTM": [nn.LSTM, 32, 64, 1],
        "GRU": [nn.GRU, 32, 64, 1]
        }
loss_fn = nn.CrossEntropyLoss()

In [ ]:
model = Model(*models["RNN"])

optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
training = Training(model, loss_fn, optimizer)
training.train(train, test)

Epoch: 1, loss: 46.0156, acc: 0.1193 | test loss: 9.1884, test acc: 0.1522 | 0.29 sec.
Epoch: 10, loss: 39.6197, acc: 0.2546 | test loss: 7.8447, test acc: 0.2487 | 0.18 sec.
Epoch: 20, loss: 14.3955, acc: 0.7417 | test loss: 2.3295, test acc: 0.7922 | 0.19 sec.
Epoch: 30, loss: 0.0523, acc: 1.0000 | test loss: 0.0101, test acc: 1.0000 | 0.19 sec.
Epoch: 40, loss: 0.0224, acc: 1.0000 | test loss: 0.0045, test acc: 1.0000 | 0.25 sec.
Epoch: 50, loss: 0.0140, acc: 1.0000 | test loss: 0.0028, test acc: 1.0000 | 0.17 sec.
Epoch: 60, loss: 0.0100, acc: 1.0000 | test loss: 0.0020, test acc: 1.0000 | 0.19 sec.


In [ ]:
model = Model(*models["LSTM"])

optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
training = Training(model, loss_fn, optimizer)
training.train(train, test)

Epoch: 1, loss: 46.0137, acc: 0.1219 | test loss: 9.1825, test acc: 0.1478 | 0.13 sec.
Epoch: 10, loss: 45.2999, acc: 0.1883 | test loss: 9.0704, test acc: 0.1804 | 0.14 sec.
Epoch: 20, loss: 42.6239, acc: 0.2264 | test loss: 8.4702, test acc: 0.2324 | 0.15 sec.
Epoch: 30, loss: 16.9334, acc: 0.6691 | test loss: 3.1767, test acc: 0.7007 | 0.15 sec.
Epoch: 40, loss: 2.0482, acc: 0.9996 | test loss: 0.3750, test acc: 0.9999 | 0.16 sec.
Epoch: 50, loss: 0.6257, acc: 1.0000 | test loss: 0.1225, test acc: 1.0000 | 0.21 sec.
Epoch: 60, loss: 0.3425, acc: 1.0000 | test loss: 0.0685, test acc: 0.9999 | 0.15 sec.


In [ ]:
model = Model(*models["GRU"])

optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
training = Training(model, loss_fn, optimizer)
training.train(train, test)

Epoch: 1, loss: 45.8506, acc: 0.1580 | test loss: 9.1341, test acc: 0.1802 | 0.15 sec.
Epoch: 10, loss: 44.9171, acc: 0.1874 | test loss: 8.9799, test acc: 0.1805 | 0.15 sec.
Epoch: 20, loss: 39.4514, acc: 0.2593 | test loss: 7.9009, test acc: 0.2558 | 0.14 sec.
Epoch: 30, loss: 4.6539, acc: 0.9977 | test loss: 0.7804, test acc: 0.9984 | 0.15 sec.
Epoch: 40, loss: 0.7407, acc: 1.0000 | test loss: 0.1430, test acc: 1.0000 | 0.14 sec.
Epoch: 50, loss: 0.3486, acc: 1.0000 | test loss: 0.0691, test acc: 1.0000 | 0.16 sec.
Epoch: 60, loss: 0.2182, acc: 1.0000 | test loss: 0.0438, test acc: 1.0000 | 0.14 sec.


In [ ]:
x, y = generate_sequences()

x = x.to(device)
y = y.to(device)
out = model.forward(x).view(-1, INPUT_DIM).argmax(dim=1)

print(f"Original sentence is : {x}")
print("-" * 100)
print(f"Encrypted sentence is : {y}")
print("-" * 100)
print(f"Predicted sentence is : {out}")

Original sentence is : tensor([[3, 8, 1, 8, 6, 7, 7, 7, 1, 2]], device='cuda:0')
----------------------------------------------------------------------------------------------------
Encrypted sentence is : tensor([[3, 1, 4, 1, 9, 0, 0, 0, 4, 5]], device='cuda:0')
----------------------------------------------------------------------------------------------------
Predicted sentence is : tensor([3, 1, 4, 1, 9, 0, 0, 0, 4, 5], device='cuda:0')
